<div class="alert alert-block alert-info"><h1 style="text-align:center;color:black"> Fashion style Classification Problem 📷 🚀 </h1> </div>
The Dogs & Cats is a foundational problem for a basic CNN(convolutional neural network) model which involves classifying images as a fashion styles.The dataset can be used for learning how to develop,evaluate and use convolutional deep learning neural networks for classification of images. This includes how to develop a robust test harness for estimating the performance of the model, exploring improvements for the model by changing the paramters of the model, saving and loading the model to make predicitions on new data.

![fashion](..//fashion_recognition/Datasets/dataset-cover.png)

<div class="alert alert-block alert-warning"><h2 style="text-align:Center;color:black">Table of Content </h2> </div>

1. [Introduction 💥](#1)
1. [Data Description](#2)
1. [Loading Libraries📖](#3)
1. [Data Extraction📁](#4)
1. [Data Exploration📊](#5)
1. [Train Test Split](#6)
    1. [Using Dataframe](#7)
    1. [Using Directory](#8)
1. [Data Preparation 🛠️](#9)
    1. [Image Data Generator](#10)
        1. [Using DataFrame](#11)
        1. [Using Directory](#12)
1. [Deep learning Model ⚙️](#13)
    1. [Model Layers](#14)
    1. [Callbacks](#15)
    1. [Compile Model](#23)
    1. [Fit Model](#16)
    1. [Plot Result](#17)
    1. [Evaluvation](#18)
1. [Prediction](#19)
    1. [Visualize Classified Images](#20)
1. [Submission](#21)
1. [Conclusion](#22)

<a id='1'></a>
<div class="alert alert-block alert-danger"><h2 style="text-align:center;color:black"> Introduction 💥</h2> </div> 

In this notebook, we will discover how to develop a CNN to classify images of fashion_styles.

We will follow this steps:

* Load and prepare the images for training purpose.
* Split data for training and validation purpose.
* Apply Data Augmentation to the data.
* Develop a CNN model using keras and how to choose various parameters for improving performance of the model.
* Evaluate performance of our model.
* Save and load a model for further predictions.
* Draw the confusion matrix for trained model.

<a id='2'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Data Description </h2> </div>

The training archive contains 44,110 images of dogs and cats.

Train your algorithm on these files and predict the labels.


<a id='3'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Loading Libraries </h2> </div>

In [ ]:
# Basic
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

from os import makedirs
from os import listdir
from shutil import copyfile
from random import seed
from random import random
import numpy as np
import pandas as pd

# visuals
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay

# Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,MaxPooling2D,Dropout,Flatten,BatchNormalization,Conv2D
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [ ]:
#Define the source dir where the images come from
source = '../fashion_recognition/Datasets/fashion-dataset/images'

# We will save in this dir the files we select to train the model
destination = '../fashion_recognition/Datasets/fashion-dataset/working'

# We will classify in this dir the images classified by syle
dataset_home = '../fashion_recognition/Datasets/fashion-dataset/train_test_styles/'

In [ ]:
makedirs(destination, exist_ok=True)
makedirs(dataset_home, exist_ok=True)

In [ ]:
#We remove files of previous executions
for filename in os.listdir(destination):
    os.remove(os.path.join(destination, filename))

In [ ]:
for dirname, _, filenames in os.walk(dataset_home):
    for filename in filenames:
        os.remove(os.path.join(dirname, filename))

<a id='4'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Data Extraction </h2> </div>

In [ ]:
files = "../fashion_recognition/Datasets/fashion-dataset/images/"

### Loading Images in a Dataframe

In [ ]:
#We create a dataframe with the info of the images in the datasets
csv_dir = "../fashion_recognition/Datasets/fashion-dataset/"

data = pd.read_csv(csv_dir + "styles.csv", sep=";")

In [ ]:
data.head()

In [ ]:
#Name of the file
data['filename_source'] = data['id'].astype(str) + '.jpg'

#Name of the style
data['label'] = data['usage']

#Origin route
data['source'] = source + '/' + data['filename_source']

#Filename in the destination route. It contains the label + id
data['filename_destination'] = data['usage'] + '.' + data['filename_source']

#Destination route
data['destination'] = destination + '/' + data['filename_destination']

#We remove the cells we don't need
data.drop(labels=['gender', 'masterCategory', 'subCategory', 'articleType',
       'baseColour', 'season', 'year', 'usage', 'productDisplayName'], axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
#We delete the registers whose label is null
data = data[data.label.notna()]

In [ ]:
#We check if we actually have an image file for the registers
images = os.listdir(source)
data['exists'] = data.filename_source.isin(images)

data = data[data['exists'] == True]

data.drop(labels='exists', axis=1, inplace=True)

In [ ]:
data.groupby('label').count()

In [ ]:
#Casual style has many registers. We select only 4100
casual_style = data[data['label'] == 'Casual'].sample(4100)

In [ ]:
#There are registers with only a few registers. We select only the styles with more than 1000
rest_styles = data[
    (data['label'] == 'Ethnic') |
    (data['label'] == 'Formal') |
    (data['label'] == 'Sports')
]

In [ ]:
data = pd.concat([casual_style, rest_styles], ignore_index=True)

In [ ]:
for i in data.index:
    src_path = data['source'][i]
    dst_path = data['destination'][i]
    copyfile(src_path, dst_path)

<a id='5'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Data Exploration </h2> </div>

First, we start by visualizing the variable of interest.

Let's view more images in a grid format.

<h5 style="text-align:center;color:Green">We visualize few images of Casual Style. </h5>

In [ ]:
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)

df_label = data[data['label'] == 'Casual'][['destination']]
df_label.reset_index(drop=True, inplace=True)

for i in range(10):

    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = df_label['destination'][i]
    image = imread(filename)
    plt.imshow(image)
    plt.title('Casual',fontsize=12)
    plt.axis('off')

plt.show()

<h5 style="text-align:center;color:Red">We visualize few images of Etnic style. </h5>

In [ ]:
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)

df_label = data[data['label'] == 'Ethnic'][['destination']]
df_label.reset_index(drop=True, inplace=True)

for i in range(10):

    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = df_label['destination'][i]
    image = imread(filename)
    plt.imshow(image)
    plt.title('Ethnic',fontsize=12)
    plt.axis('off')

plt.show()

<h5 style="text-align:center;color:Red">We visualize few images of Sports style. </h5>

In [ ]:
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)

df_label = data[data['label'] == 'Sports'][['destination']]
df_label.reset_index(drop=True, inplace=True)

for i in range(10):

    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = df_label['destination'][i]
    image = imread(filename)
    plt.imshow(image)
    plt.title('Sports',fontsize=12)
    plt.axis('off')

plt.show()

<h5 style="text-align:center;color:Red">We visualize few images of Formal style. </h5>

In [ ]:
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.subplots_adjust(hspace=0.4)

df_label = data[data['label'] == 'Formal'][['destination']]
df_label.reset_index(drop=True, inplace=True)

for i in range(10):

    plt.subplot(1,10,i+1)    # the number of images in the grid is 10*10 (100)
    filename = df_label['destination'][i]
    image = imread(filename)
    plt.imshow(image)
    plt.title('Formal',fontsize=12)
    plt.axis('off')

plt.show()

Images are of varying size.

Every time the cell is run different set of images will be displayed, one can scan the images of both categories. Presence of humans in some images could be a challenge for the model to classify.

Some images have more than one cats or dogs respectively.

<a id='6'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Train Test Split </h2> </div>

<a id='7'></a>
<div class="alert alert-block alert-warning"><h4 style="text-align:center;color:black"> Using Dataframe </h4> </div>

In [ ]:
# train test split using dataframe

labels = data['label']

X_train, X_temp = train_test_split(data, test_size=0.2, stratify=labels, random_state = 42)

label_test_val = X_temp['label']

X_test, X_val = train_test_split(X_temp, test_size=0.5, stratify=label_test_val, random_state = 42)

print('The shape of train data',X_train.shape)
print('The shape of test data',X_test.shape)
print('The shape of validation data',X_val.shape)

Now we will Create a barplot to see the class distrubtion in trainting dataset.

In [ ]:
labels = list(data.label.unique())

label1,count1 = np.unique(X_train.label,return_counts=True)
label2,count2 = np.unique(X_val.label,return_counts=True)
label3,count3 = np.unique(X_test.label,return_counts=True)

uni1 = pd.DataFrame(data=count1,index=labels,columns=['Count1'])
uni2 = pd.DataFrame(data=count2,index=labels,columns=['Count2'])
uni3 = pd.DataFrame(data=count3,index=labels,columns=['Count3'])


plt.figure(figsize=(20,6),dpi=200)
sns.set_style('darkgrid')

plt.subplot(131)
sns.barplot(data=uni1,x=uni1.index,y='Count1',palette='icefire',width=0.2).set_title('Class distribution in Training set',fontsize=15)
plt.xlabel('Labels',fontsize=12)
plt.ylabel('Count',fontsize=12)

plt.subplot(132)
sns.barplot(data=uni2,x=uni2.index,y='Count2',palette='icefire',width=0.2).set_title('Class distribution in validation set',fontsize=15)
plt.xlabel('Labels',fontsize=12)
plt.ylabel('Count',fontsize=12)


plt.subplot(133)
sns.barplot(data=uni3,x=uni3.index,y='Count3',palette='icefire',width=0.2).set_title('Class distribution in Testing set',fontsize=15)
plt.xlabel('Labels',fontsize=12)
plt.ylabel('Count',fontsize=12)

plt.show()

+<a id='8'></a>
<div class="alert alert-block alert-warning"><h4 style="text-align:center;color:black"> Using Directory </h4> </div>

In [ ]:
# create directories
subdirs = ['train/', 'test/']

In [ ]:
for subdir in subdirs:
    # create label subdirectories
    labeldirs = []
    for label in labels:
        labeldirs.append(label + '/')
    for labldir in labeldirs:
        newdir = dataset_home + subdir + labldir
        makedirs(newdir, exist_ok=True)

In [ ]:
# seed random number generator
seed(1)
# define ratio of pictures to use for validation
val_ratio = 0.2

In [ ]:
# copy training dataset images into subdirectories
src_directory = '../fashion_recognition/Datasets/fashion-dataset/working'
for file in listdir(src_directory):
        src = src_directory + '/' + file
        dst_dir = 'train/'
        if random() < val_ratio:
            dst_dir = 'test/'
        if file.startswith('Casual'):
            dst = dataset_home + dst_dir + 'Casual/' + file
            copyfile(src, dst)
        elif file.startswith('Ethnic'):
            dst = dataset_home + dst_dir + 'Ethnic/' + file
            copyfile(src, dst)
        elif file.startswith('Formal'):
            dst = dataset_home + dst_dir + 'Formal/' + file
            copyfile(src, dst)
        elif file.startswith('Sports'):
            dst = dataset_home + dst_dir + 'Sports/' + file
            copyfile(src, dst)

In [ ]:
path1 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/train/Casual"
path2 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/train/Ethnic"
path3 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/train/Formal"
path4 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/train/Sports"
path5 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/test/Casual"
path6 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/test/Ethnic"
path7 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/test/Formal"
path8 = "../fashion_recognition/Datasets/fashion-dataset/train_test_styles/test/Sports"


print('Then number of Casual images in training data is' ,len(os.listdir(path1)))
print('Then number of Ethnic images in training data is' ,len(os.listdir(path2)))
print('Then number of Formal images in training data is' ,len(os.listdir(path3)))
print('Then number of Sports images in training data is' ,len(os.listdir(path4)))
print('Then number of Casual images in validation data is' ,len(os.listdir(path5)))
print('Then number of Ethnic images in validation data is' ,len(os.listdir(path6)))
print('Then number of Formal images in validation data is' ,len(os.listdir(path7)))
print('Then number of Sports images in validation data is' ,len(os.listdir(path8)))


<a id='9'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Data Preparation </h2> </div>

firstly, we will list out all the important parameters and respective values.

In [ ]:
# parameters
image_size = 128
image_channel = 3
bat_size = 32

<a id='10'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black">Image Data Generator </h3> </div>

* The data for will used by flow_from_dataframe and flow_from_directory.
* The batch size is 32 and the image size is (128,128).


In [ ]:
# Creating image data generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                    rotation_range = 15,
                                    horizontal_flip = True,
                                    zoom_range = 0.2,
                                    shear_range = 0.1,
                                    fill_mode = 'reflect',
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1)

test_datagen = ImageDataGenerator(rescale=1./255)

<a id='11'></a>
<div class="alert alert-block alert-warning"><h4 style="text-align:center;color:black"> Using Dataframe </h4> </div>

In [ ]:
# Applying image data gernerator to train and test data

train_generator = train_datagen.flow_from_dataframe(X_train,
                                                    directory = '../fashion_recognition/Datasets/fashion-dataset/working',
                                                    x_col= 'filename_destination',
                                                    y_col= 'label',
                                                    batch_size = bat_size,
                                                    target_size = (image_size,image_size)
                                                   )
val_generator = test_datagen.flow_from_dataframe(X_val, 
                                                 directory = '../fashion_recognition/Datasets/fashion-dataset/working',
                                                 x_col= 'filename_destination',
                                                 y_col= 'label',
                                                 batch_size = bat_size,
                                                 target_size = (image_size,image_size),
                                                 shuffle=False
                                                )

test_generator = test_datagen.flow_from_dataframe(X_test, 
                                                  directory = '../fashion_recognition/Datasets/fashion-dataset/working',
                                                  x_col= 'filename_destination',
                                                  y_col= 'label',
                                                  batch_size = bat_size,
                                                  target_size = (image_size,image_size),
                                                  shuffle=False
                                                 )

<a id='12'></a>
<div class="alert alert-block alert-warning"><h4 style="text-align:center;color:black"> Using Directory </h4> </div>

In [ ]:
train_gen = train_datagen.flow_from_directory('../fashion_recognition/Datasets/fashion-dataset/train_test_styles/train/',
                                              class_mode='binary',
                                              target_size = (image_size,image_size),
                                              batch_size = bat_size,
                                             )

val_gen = test_datagen.flow_from_directory('../fashion_recognition/Datasets/fashion-dataset/train_test_styles/test/',
                                          class_mode='binary',
                                          batch_size = bat_size,
                                          target_size = (image_size,image_size),
                                          shuffle = False
                                         )

<a id='13'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Deep Learning Model </h2> </div>

<a id='14'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black">Model Layers </h3> </div>


### Architecture

The architecture of the Cat vs Dog Image Classification model consists of the following Layers and components:

#### Layers :
* The input layer consist of a Conv2D with 32 filters and activation relu.
* The model contain the 3 blocks of convolution with increasing filters and activation relu.
* Each convolution block contains Batch Noramlization, Max pooling (pool_size = 2) and Dropout (0.2).
* The fully connected layers contain Flatten layer, Dense layer with 512 units and a Dropout layer.
* The output layer is a Dense layer with 2 units and softmax activation.

#### Components:

* **Input Layer:** Receives input images for classification.
* **Convolutional Layers:** Extract features from the images through convolutional operations.
* **Pooling Layers:** Reduce the spatial dimensions of the feature maps.
* **Flatten Layer:** Convert the 2D feature maps into a 1D vector.
* **Fully Connected Layers:** Perform classification using densely connected layers.
* **Output Layer:** Provides the final prediction probabilities for each fashion style classes.

In [ ]:
model = Sequential()

# Input Layer
model.add(Conv2D(32,(3,3),activation='relu',input_shape = (image_size,image_size,image_channel))) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Bloack 1 
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# Block 2
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# Block 3
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Fully Connected layers 
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# Output layer
model.add(Dense(4,activation='softmax'))

model.summary()

<a id='15'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Callbacks </h3> </div>
we will be using two callbacks -

* **ReduceLROnPlateau :** Reduce learning rate when a metric has stopped improving.
* **EarlyStopping :** Stop training when a monitored metric has stopped improving.

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience=2,
                                            factor=0.5,
                                            min_lr = 0.00001,
                                            verbose = 1)

early_stoping = EarlyStopping(monitor='val_loss',patience= 3,restore_best_weights=True,verbose=0)

<a id='23'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Compile the model </h3> </div>
Finally we will compile the model .There are 3 things to mention here : Optimizer,Loss, Metrics

* **Optimizer** :- To minimize cost function we use different methods For ex :- like gradient descent, stochastic gradient descent. So these are call optimizers. We are using a default one here which is adam.
​
* **Loss** :- To make our model better we either minimize loss or maximize accuracy. Neural Networks always minimize loss. To measure it we can use different formulas like 'categorical_crossentropy' or 'binary_crossentropy'. Here I have used binary_crossentropy.
​
* **Metrics** :- This is to denote the measure of your model. Can be accuracy or some other metric.

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

<a id='16'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Fit the model </h3> </div>

We are now going to train our compiled model using the train iterator (train_generator) and use the val iterator (val_generator) as a validation dataset during training.

The number of steps for the train and validation iterators must be specified. This is the number of batches that will comprise one epoch. This can be specified via the length of each iterator, and will be the total number of images in the train and validation directories divided by the batch size (32).

The model will be fit for 30 epochs.

In [ ]:
styles_model = model.fit(train_generator,
                         validation_data = val_generator,
                         callbacks=[early_stoping,learning_rate_reduction],
                         epochs = 30,
                         # steps_per_epoch = len(train_generator),
                         # validation_steps = len(val_generaotor),
                         )

<a id='17'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Plot the results </h3> </div>

In [ ]:
# plots for accuracy and Loss with epochs

error = pd.DataFrame(styles_model.history)

plt.figure(figsize=(18,5),dpi=200)
sns.set_style('darkgrid')

plt.subplot(121)
plt.title('Cross Entropy Loss',fontsize=15)
plt.xlabel('Epochs',fontsize=12)
plt.ylabel('Loss',fontsize=12)
plt.plot(error['loss'])
plt.plot(error['val_loss'])

plt.subplot(122)
plt.title('Classification Accuracy',fontsize=15)
plt.xlabel('Epochs',fontsize=12)
plt.ylabel('Accuracy',fontsize=12)
plt.plot(error['accuracy'])
plt.plot(error['val_accuracy'])

plt.show()

<a id='18'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Evaluvation </h3> </div>

we will evaluvate the Training and validation data accuracy and loss.

In [ ]:
# Evaluvate for train generator
loss,acc = model.evaluate(train_generator,batch_size = bat_size, verbose = 0)

print('The accuracy of the model for training data is:',acc*100)
print('The Loss of the model for training data is:',loss)

# Evaluvate for validation generator
loss,acc = model.evaluate(val_generator,batch_size = bat_size, verbose = 0)

print('The accuracy of the model for validation data is:',acc*100)
print('The Loss of the model for validation data is:',loss)

Finally, we will save the model for future use.

In [ ]:
# Save the Model
model.save("model.h5")

<a id='19'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Prediction </h2> </div>

Now, we will predict the model on test dataset.

In [ ]:
# prediction
result = model.predict(test_generator,batch_size = bat_size,verbose = 0)

y_pred = np.argmax(result, axis = 1)

y_true = test_generator.labels

# Evaluvate
loss,acc = model.evaluate(test_generator, batch_size = bat_size, verbose = 0)

print('The accuracy of the model for testing data is:',acc*100)
print('The Loss of the model for testing data is:',loss)

Classification report

In [ ]:
print(classification_report(y_true, y_pred,target_names=labels))

confusioin matrix

In [ ]:
confusion_mtx = confusion_matrix(y_true,y_pred) 

f,ax = plt.subplots(figsize = (8,4),dpi=200)
sns.heatmap(confusion_mtx, annot=True, linewidths=0.1, cmap = "gist_yarg_r", linecolor="black", fmt='.0f', ax=ax,cbar=False, xticklabels=labels, yticklabels=labels)

plt.xlabel("Predicted Label",fontsize=10)
plt.ylabel("True Label",fontsize=10)
plt.title("Confusion Matrix",fontsize=13)

plt.show()

<a id='20'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> submission </h2> </div>
Make predictions on kaggle test data for submission.

In [ ]:
size =(128,128)

# loading into dataframe
test_dir = "../fashion_recognition/working/test1/"
filenames = os.listdir(test_dir)
test_data = pd.DataFrame({"filename": filenames})
test_data['label'] = 'unknown'

In [ ]:


# Create data genenerator for test data
test1_idg =  test_datagen.flow_from_dataframe(test_data, 
                                     "../fashion_recognition/working/test1/",
                                     x_col= "filename",
                                     y_col = 'label',
                                     batch_size = bat_size,
                                     target_size=size, 
                                     shuffle = False)

# Test Prediction
test1_predict = model.predict(test1_idg,verbose = 0)

test1_predict_argmax = np.argmax(test1_predict, axis=1)

y_test_pred = test1_predict_argmax

test_data['label'] = y_test_pred

# mapping
label_mapping = {0: 'cat', 1: 'dog'}
test_data['label'] = test_data['label'].map(label_mapping)
test_data.head()

# csv file output for submission
sub = pd.read_csv('../fashion_recognition/Datasets/sampleSubmission.csv',index_col='id')

sub['label'] = y_test_pred

sub.to_csv('../fashion_recognition/Datasets/submission.csv',index=True)

<a id='21'></a>
<div class="alert alert-block alert-danger"><h3 style="text-align:center;color:black"> Visualize Classified Images  </h3> </div>

In [ ]:
fig, axes = plt.subplots(1, 10, figsize=(20, 4))
for idx in range(10):
    image_path = os.path.join(test_dir, test_data.iloc[idx]['filename'])
    image = Image.open(image_path)
    axes[idx].imshow(image)
    axes[idx].set_title("Label: " + test_data.iloc[idx]['label'])
    axes[idx].axis('off')
plt.show()

After watching the images we can see the our model is quite accurate with 94% accuracy.


<a id='22'></a>
<div class="alert alert-block alert-info"><h2 style="text-align:center;color:black"> Conclusion </h2> </div>


We successfully built a deep neural network model by implementing Convolutional Neural Network (CNN) to classify dog and cat images with very high accuracy 97.32 %. 

The model was used to predict the classes of the images from the independent test set and results were submitted to test the accuracy of the prediction with fresh data.

The Cat vs Dog Image Classification model demonstrates the successful implementation of a Convolutional Neural Network for image classification tasks. By accurately distinguishing between images of cats and dogs, this project showcases the potential of deep learning algorithms in solving real-world problems involving image analysis. Through this project, we aim to inspire further exploration of CNNs and their applications in various domains,